# VacationPy
---

## Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,tralee,52.2704,-9.7026,9.14,100,40,2.06,IE,1700992017
1,port-aux-francais,-49.3500,70.2167,4.10,69,42,10.56,TF,1700992329
2,edinburgh of the seven seas,-37.0676,-12.3116,14.29,79,76,12.27,SH,1700992158
3,tera,14.0078,0.7531,31.65,10,52,4.00,NE,1700992332
4,east ballina,-28.8667,153.5833,20.78,90,95,4.34,AU,1700992334


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [15]:
%%capture --no-display

# Configure the map plot
map_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=800,
    frame_height=600,
    color="City",
    size="Humidity"
)

# Display the map
%capture c --no-display 
map_1

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
idealweather_df= city_data_df.loc[(city_data_df["Cloudiness"]<=0)
                                  &(city_data_df["Max Temp"]<27 )
                                  &(city_data_df["Max Temp"]>21)
                                  &(city_data_df["Wind Speed"]<5)
                                  &(city_data_df["Humidity"]<80)]    



# Drop any rows with null values
idealweather_df.dropna()

# Display sample data
idealweather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,limoeiro do norte,-5.1456,-38.0981,26.70,57,0,4.26,BR,1700992354
131,mahina,-17.5065,-149.4890,25.87,73,0,2.57,PF,1700992242
211,vila velha,-20.3297,-40.2925,22.19,77,0,4.02,BR,1700992701
225,sari,36.5633,53.0601,21.77,56,0,3.09,IR,1700992726
305,taoudenni,22.6783,-3.9836,21.24,18,0,2.75,ML,1700992925
332,bundu,23.1833,85.5833,25.12,27,0,2.17,IN,1700992975
347,tura,25.5198,90.2201,24.20,61,0,1.18,IN,1700993005
469,arar,30.9753,41.0381,26.01,24,0,2.57,SA,1700993219


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
 
hotel_df=idealweather_df[["City","Country","Lat","Lng","Humidity"]]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

#hotel_df["Hotel Name"]="-----"
hotel_df.insert(5, "Hotel Name", "------") 

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
14,limoeiro do norte,BR,-5.1456,-38.0981,57,------
131,mahina,PF,-17.5065,-149.4890,73,------
211,vila velha,BR,-20.3297,-40.2925,77,------
225,sari,IR,36.5633,53.0601,56,------
305,taoudenni,ML,22.6783,-3.9836,18,------
332,bundu,IN,23.1833,85.5833,27,------
347,tura,IN,25.5198,90.2201,61,------
469,arar,SA,30.9753,41.0381,24,------


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
categories="accommodation.hotel"
params = {"apiKey":geoapify_key,
          "categories":categories}
         

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    Latitude=row["Lat"]
    Longitude=row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Longitude},{Latitude},{radius}"
    params["bias"] = f"proximity:{Longitude},{Latitude}"
    
    # Set base URL (provided)
    base_url = "https://api.geoapify.com/v2/places"
    


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url,params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
   
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
limoeiro do norte - nearest hotel: Clasaic Hotel
mahina - nearest hotel: Motu Martin
vila velha - nearest hotel: Hotel Prainha
sari - nearest hotel: گراند هتل
taoudenni - nearest hotel: No hotel found
bundu - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
arar - nearest hotel: دار أوتيل Dar Otel


,City,Country,Lat,Lng,Humidity,Hotel Name
14,limoeiro do norte,BR,-5.1456,-38.0981,57,Clasaic Hotel
131,mahina,PF,-17.5065,-149.4890,73,Motu Martin
211,vila velha,BR,-20.3297,-40.2925,77,Hotel Prainha
225,sari,IR,36.5633,53.0601,56,گراند هتل
305,taoudenni,ML,22.6783,-3.9836,18,No hotel found
332,bundu,IN,23.1833,85.5833,27,No hotel found
347,tura,IN,25.5198,90.2201,61,No hotel found
469,arar,SA,30.9753,41.0381,24,دار أوتيل Dar Otel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width=800,
    frame_height=600,
    color="City",
    size="Humidity",
    hover_cols = ["City", "Country","Lat","Lng","Humidity","Hotel Name"]
    )

# Display the map
%capture c --no-display 
hotel_map